In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm import tqdm

In [14]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

In [15]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=20, shuffle=True, num_workers=2,  pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=20, shuffle=True, num_workers=2, pin_memory=True)

In [30]:
class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(1, 16, 3),
        nn.Dropout2d(0.2),
        nn.ReLU(),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(16, 32, 3),
        nn.Dropout2d(0.2),
        nn.ReLU(),

        nn.Flatten()
    )
    self.classifer = nn.Sequential(
        nn.Linear(32 * 11 * 11, 512),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(512, 128),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.Linear(128, 10),
        nn.Dropout(0.2),
        nn.ReLU(),

        nn.LogSoftmax(dim=1)
    )


    self.loss = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0008, betas=(0.9, 0.999))
    self.scheduler = None

  def forward(self, x):
    x = self.features(x)
    return self.classifer(x)

  def train_model(self, training_loader, epoch):
    self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=0.002, steps_per_epoch=len(training_loader), epochs=epoch)
    self.train()
    self.to('cuda')
    for i in tqdm(range(epoch)):
      running_loss=0
      for images, labels in training_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        self.optimizer.zero_grad()
        pred = self.forward(images)
        loss = self.loss(pred, labels)
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        running_loss += loss.item()


      print(running_loss/len(training_loader))


  def test_model(self, testing_loader):
      self.eval()
      self.to('cuda')
      true = 0
      with torch.no_grad():
        for images, labels in testing_loader:
          images = images.to('cuda')
          labels = labels.to('cuda')
          pred = self.forward(images)
          pred_actual = torch.argmax(pred, dim=-1)
          true += (pred_actual == labels).sum()
        print(true)
        print(len(testing_loader)*20)
        print(true/(len(testing_loader)*20))



In [31]:
print(f"CUDA Available: {torch.cuda.is_available()}")
model = Model().to('cuda')
model.train_model(training_loader, 20)
model.test_model(testing_loader)


CUDA Available: True


  5%|████▏                                                                              | 1/20 [00:10<03:14, 10.25s/it]

1.1431470801830292


 10%|████████▎                                                                          | 2/20 [00:20<03:05, 10.28s/it]

0.9165615956683953


...

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:13<00:10, 10.14s/it]

0.5880601193010807


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:23<00:00, 10.19s/it]

0.5790830375454776


tensor(9248, device='cuda:0')
10000
tensor(0.9248, device='cuda:0')
